In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt

Json_prompt = load_prompt("./prompt.json")
Yaml_prompt = load_prompt("./prompt.yaml")
chat = ChatOpenAI(
    temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]
)

Json_prompt.format(country="Germany")
Yaml_prompt.format(country="Greece")


'What is the captial of Greece'